In [1]:
 import basilica

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_csv('data/merged_df.csv')

In [5]:
df = df.fillna('none')

In [6]:
df['combined_text'] = df.name + " " + df.flavors +  " " + df.race + " " + df.positive + " " + df.negative + " " + df.medical + " " + df.Description

In [7]:
key='2386d269-f7e1-7126-6045-8b52f9192e03'

In [8]:
with basilica.Connection(key) as c:
    embedded = []
    for row in df['combined_text']:
        sentence = row
        embedding = list(c.embed_sentence(sentence))
        embedded.append(embedding)
    df['embedded'] = embedded

In [9]:

data_input = np.stack(df['embedded'].values, axis=0)

In [10]:
dtm = pd.DataFrame(data_input)

In [11]:
from sklearn.neighbors import NearestNeighbors

In [12]:
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [13]:
user_input = " help with pain and stress but has a sweet flavor"
with basilica.Connection(key) as c:
    embedded = c.embed_sentence(user_input)

embedded = np.stack([embedded], axis=0)

In [14]:
score,strain= nn.kneighbors(embedded)

In [15]:
strain

array([[ 22, 148, 110, 311, 302]])

In [18]:
strains = [df[['name', 'medical','flavors']].loc[n] for n in strain]

In [19]:
strains

[          name                                            medical  \
 22      Avalon           Insomnia, Pain, Stress, Nausea, Headache   
 148      Gumbo  Depression, Insomnia, Pain, Stress, Headache, ...   
 110     Dopium  Depression, Pain, Stress, Nausea, Headache, He...   
 311    Trident  Depression, Pain, Stress, Inflammation, Muscle...   
 302  Tangilope  Depression, Pain, Stress, Lack of Appetite, Fa...   
 
                              flavors  
 22              Earthy, Nutty, Sweet  
 148  Earthy, Sweet, Pungent, Flowery  
 110            Pine, Pungent, Earthy  
 311   Pungent, Earthy, Citrus, Sweet  
 302  Sweet, Citrus, Orange, Tropical  ]